In [ ]:
'''2.Build a Multiclass classifier using the CNN model. Use
MNIST or any other suitable dataset. a. Perform Data Pre- processing b. Define Model and perform training c. Evaluate
Results using confusion matrix.'''

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns # For plotting the confusion matrix nicely
import matplotlib.pyplot as plt


In [ ]:
# --- Configuration ---
batch_size = 128
epochs = 5 # Keep low for quick demonstration
num_classes = 10
input_shape = (28, 28, 1) # MNIST images are 28x28 grayscale

# --- a. Perform Data Pre-processing ---
print("Loading and pre-processing MNIST data...")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


In [ ]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)


In [ ]:

# Convert class vectors to binary class matrices (one-hot encoding)
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)
print(f"x_train shape: {x_train.shape}")
print(f"{x_train.shape[0]} train samples, {x_test.shape[0]} test samples")


# --- b. Define Model and Perform Training ---
print("\nBuilding the CNN model...")
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5), # Dropout for regularization
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary() # Print model architecture


In [ ]:

print("\nCompiling and training the model...")
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(x_train, y_train_cat, batch_size=batch_size, epochs=epochs, validation_split=0.1)



In [ ]:
# --- c. Evaluate Results using Confusion Matrix ---
print("\nEvaluating the model on the test set...")
score = model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]:.4f}")

# Get predictions for the test set
y_pred_probabilities = model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probabilities, axis=1) # Get class index with highest probability


In [ ]:

# Generate the confusion matrix
print("\nGenerating Confusion Matrix...")
cm = confusion_matrix(y_test, y_pred_classes) # Compare true labels (y_test) with predicted labels

# Optional: Display the confusion matrix nicely using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Optional: Print classification report for more metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=[str(i) for i in range(num_classes)]))
